# Synthesize Model with FINN
In this part of the demo we're going to load the model we exported in the last step and run it through FINN. Afterwards we'll copy the generated bitfile into the pynq-deployment folder and zip it so that you can upload it to your PYNQ board.

In [1]:
import os
import shutil
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg

In [2]:
model_dir = "./onnx"
model_file = model_dir + "/model_s3_finn.onnx"

out_dir = "./outputs/example_output"

In [3]:
#Delete previous run results if exist
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
    print("Previous run results deleted!")

In [4]:
# I'm using the Arty Z720 board. However, larger devices can achieve higher throughput (e.g. Zynq Ultrascale+)
cfg = build.DataflowBuildConfig(
    output_dir=out_dir,
    mvau_wwidth_max=36,
    target_fps=150,
    synth_clk_period_ns=11.0,
    board="Pynq-Z2",
    fpga_part="xc7z020clg400-1",
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE
    ],
    save_intermediate_models=True,
)

In [5]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from ./onnx/model_s3_finn.onnx
Intermediate outputs will be generated in /tmp/finn_dev_dan
Final outputs will be generated in ./outputs/example_output
Build log is at ./outputs/example_output/build_dataflow.log
Running step: step_qonnx_to_finn [1/19]
Running step: step_tidy_up [2/19]
Running step: step_streamline [3/19]
Running step: step_convert_to_hw [4/19]
Running step: step_create_dataflow_partition [5/19]
Running step: step_specialize_layers [6/19]
Running step: step_target_fps_parallelization [7/19]
Running step: step_apply_folding_config [8/19]
Running step: step_minimize_bit_width [9/19]
Running step: step_generate_estimate_reports [10/19]
Running step: step_hw_codegen [11/19]
Running step: step_hw_ipgen [12/19]
Running step: step_set_fifo_depths [13/19]
Running step: step_create_stitched_ip [14/19]
Running step: step_measure_rtlsim_performance [15/19]
Running step: step_out_of_context_synthesis [16/19]
Running step: step_synthesize_bitfile [17/19]

0

# Run on PYNQ
Now we're going to copy the bitfile we just generated and move it into the pynq-deployment directory I've created. Once we've done this we can zip it, upload it to the jupyter_notebooks directory on your pynq device, unzip it and follow the last notebook. For more info on PYNQ I would recommend [this tutorial](https://www.hackster.io/90432/programming-python-on-zynq-fpga-ec4712).

In [6]:
# Show the output directory
! ls $out_dir

auto_folding_config.json  intermediate_models
bitfile			  report
build_dataflow.log	  template_specialize_layers_config.json
final_hw_config.json	  time_per_step.json


In [7]:
bitfile_path = out_dir + "/bitfile/"

In [ ]:
# Copy the bitfile directory into the pynq-deployment folder
! cp -r $bitfile_path ./pynq-deployment/your-bitfile/

In [9]:
# Zip the pynq-deployment directory and upload it to your pynq device
! zip -r pynq-deployment.zip ./pynq-deployment

updating: pynq-deployment/ (stored 0%)
updating: pynq-deployment/finn/ (stored 0%)
updating: pynq-deployment/finn/util/ (stored 0%)
updating: pynq-deployment/finn/util/data_packing.py (deflated 70%)
updating: pynq-deployment/finn/util/__init__.py (stored 0%)
updating: pynq-deployment/data/ (stored 0%)
updating: pynq-deployment/data/subject3_test_Y.npy (deflated 87%)
updating: pynq-deployment/data/subject3_test_X.npy (deflated 89%)
updating: pynq-deployment/driver.py (deflated 59%)
updating: pynq-deployment/driver_base.py (deflated 74%)
updating: pynq-deployment/example-bitfile/ (stored 0%)
updating: pynq-deployment/example-bitfile/finn-accel.bit (deflated 76%)
updating: pynq-deployment/example-bitfile/finn-accel.hwh (deflated 91%)
updating: pynq-deployment/bitfile/ (stored 0%)
updating: pynq-deployment/bitfile/finn-accel.bit (deflated 76%)
updating: pynq-deployment/bitfile/finn-accel.hwh (deflated 91%)
updating: pynq-deployment/runtime_weights/ (stored 0%)
updating: pynq-deployment/qon

Once you have the pynq zip file, upload it to your jupyter notebooks directory and unzip it. Then navigate to the 3_validate-model-with-pynq.ipynb notebook to test the accelerator on your device